# McCulloch–Pitts (MP) Neuron — Implementation Notebook

This notebook implements the **McCulloch–Pitts neuron (1943)**, a simple binary neuron model that can realize basic **logic gates** using a **threshold** rule.

**Core idea**

- Inputs are binary: $x_i \in \{0,1\}$
- Output is binary: $y \in \{0,1\}$
- Compute a weighted sum plus bias (or compare to a threshold)
- Apply a **step** (Heaviside) activation

We’ll also include the classic **inhibitory input** behavior often shown with MP neurons:

- If **any inhibitory input is 1**, the neuron **does not fire** (output 0), regardless of excitatory inputs.



## 1) Step activation and MP neuron rule

We will use:

$$
z = \sum_i w_i x_i + b
$$

$$
y =
\begin{cases}
1 & \text{if } z \ge 0 \\
0 & \text{otherwise}
\end{cases}
$$

Equivalent threshold form:

$$
\sum_i w_i x_i \ge \theta \Rightarrow y=1
$$

where $\theta$ is a threshold. We'll support both styles in code.


In [1]:
# Compatible with both Python 3 and Python 2 style printing (no f-strings)

def step(z):
    """Heaviside step function returning 0/1."""
    return 1 if z >= 0 else 0


## 2) McCulloch–Pitts neuron implementation

This implementation supports:
- **excitatory inputs**: contribute to the sum
- **inhibitory inputs**: if any inhibitory input is 1 → output is forced to 0
- either **(weights + bias)** or **(weights + threshold)** style


In [2]:
class McCullochPittsNeuron(object):
    def __init__(self, w, b=0.0, threshold=None, inhibitory_idx=None):
        self.w = list(w)
        self.b = b
        self.threshold = threshold
        self.inhibitory_idx = list(inhibitory_idx) if inhibitory_idx is not None else []

    def net_input(self, x):
        # Basic validation
        if len(x) != len(self.w):
            raise ValueError("x and w must have the same length. Got len(x)=%d, len(w)=%d" % (len(x), len(self.w)))

        # Inhibitory override: if any inhibitory input is 1 -> output must be 0
        for j in self.inhibitory_idx:
            if x[j] == 1:
                # Return a negative value so step() becomes 0
                return -1e9

        s = 0.0
        for wi, xi in zip(self.w, x):
            s += wi * xi

        if self.threshold is not None:
            # Fire if s >= threshold  <=>  s - threshold >= 0
            return s - self.threshold
        else:
            return s + self.b

    def predict(self, x):
        return step(self.net_input(x))


## 3) Helper: evaluate a neuron on all binary inputs

For 2 inputs, there are 4 patterns: (0,0), (0,1), (1,0), (1,1).  
We’ll generate a truth table for your neuron.


In [3]:
def truth_table_2(neuron):
    X = [(0,0), (0,1), (1,0), (1,1)]
    rows = []
    for x1, x2 in X:
        y = neuron.predict([x1, x2])
        rows.append((x1, x2, y))
    return rows

def print_truth_table(rows, title=None):
    if title:
        print(title)
    print("x1 x2 | y")
    print("---------")
    for x1, x2, y in rows:
        print("%d  %d  | %d" % (x1, x2, y))


## 4) Implement logic gates with an MP neuron

### AND gate
Fire only when both inputs are 1:
- weights: \([1,1]\)
- threshold: \(2\)  (since sum must be at least 2)

### OR gate
Fire when at least one input is 1:
- weights: \([1,1]\)
- threshold: \(1\)

### NAND gate
NAND is NOT(AND). Using bias style:
- weights: \([-1,-1]\)
- bias: \(1.5\)

Check:  
- if (1,1): z = -2 + 1.5 = -0.5 → 0  
- otherwise: z ≥ 0 → 1


In [4]:
# AND
and_neuron = McCullochPittsNeuron(w=[1,1], threshold=2)
print_truth_table(truth_table_2(and_neuron), title="AND gate (MP neuron)")

print()

# OR
or_neuron = McCullochPittsNeuron(w=[1,1], threshold=1)
print_truth_table(truth_table_2(or_neuron), title="OR gate (MP neuron)")

print()

# NAND
nand_neuron = McCullochPittsNeuron(w=[-1,-1], b=1.5)
print_truth_table(truth_table_2(nand_neuron), title="NAND gate (MP neuron)")


AND gate (MP neuron)
x1 x2 | y
---------
0  0  | 0
0  1  | 0
1  0  | 0
1  1  | 1

OR gate (MP neuron)
x1 x2 | y
---------
0  0  | 0
0  1  | 1
1  0  | 1
1  1  | 1

NAND gate (MP neuron)
x1 x2 | y
---------
0  0  | 1
0  1  | 1
1  0  | 1
1  1  | 0


## 5) NOT gate with a single input

NOT(x) outputs 1 when x=0, else 0.

One way:
- weight: \([-1]\)
- bias: \(0.5\)

Check:
- x=0 → z=0.5 → 1
- x=1 → z=-0.5 → 0


In [5]:
not_neuron = McCullochPittsNeuron(w=[-1], b=0.5)

for x in [0,1]:
    print("x=%d -> y=%d" % (x, not_neuron.predict([x])))


x=0 -> y=1
x=1 -> y=0


## 6) Inhibitory inputs (classic MP behavior)

Often the MP neuron is presented with:
- **excitatory** inputs: encourage firing
- **inhibitory** inputs: prevent firing if active

Example: “Fire if x1 is 1 **unless** inhibitor x2 is 1”

- Make x2 inhibitory
- Let x1 be excitatory with threshold 1
- If x2=1, output forced to 0


In [6]:
# x = [x1, x2] where x2 is inhibitory
fire_unless_inhibited = McCullochPittsNeuron(w=[1, 0], threshold=1, inhibitory_idx=[1])

print_truth_table(truth_table_2(fire_unless_inhibited), title="Fire if x1=1 unless x2 (inhibitor)=1")


Fire if x1=1 unless x2 (inhibitor)=1
x1 x2 | y
---------
0  0  | 0
0  1  | 0
1  0  | 1
1  1  | 0


## 7) Build XOR using a small MP network (2-layer construction)

One standard construction:

$$
\text{XOR}(x_1,x_2) = (x_1 \lor x_2) \land \neg(x_1 \land x_2)
$$

We’ll implement:

1) $a = OR(x_1,x_2)$  
2) $b = AND(x_1,x_2)$  
3) $c = NOT(b)$  
4) $y = AND(a,c)$


## 8) Practice exercises (students)

**Exercise A:** Create an MP neuron for **NOR** and print its truth table.

Hint: NOR is NOT(OR).

**Exercise B:** Create an MP neuron that fires if **at least 2 out of 3** inputs are 1.

Hint: weights \([1,1,1]\), threshold \(2\).

**Exercise C:** For the neuron \(w=[2,-1],\; b=-0.5\), compute predictions for:
- \([0,0]\), \([0,1]\), \([1,0]\), \([1,1]\)


In [7]:
# Exercise A: NOR gate (fill in)
# 1) Build OR neuron
# 2) Build NOT neuron
# 3) Compose NOR = NOT(OR)

# TODO: write your code here


In [8]:
# Exercise B: "2 out of 3" neuron (fill in)

# TODO: write your code here


In [9]:
# Exercise C: compute predictions (fill in)

# TODO: write your code here
